In [4]:
import pandas as pd
import numpy as np
import os
import pickle as pkl
import h5py
import sys
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.stats import kstest,ks_2samp
import scipy
import glob

In [10]:
data_dir = "./data/clusterdata-2011-2/"

task_events_header = ['timestamp','missing_info', 'job_id','task_index','machine_id','event_type',
                      'user_name','sched_class','priority','cpu_req','ram_req','space_req','diff_machine']

machine_events_header = ['timestamp','machine_id','event_type','platform_id','cpu_cap','mem_cap']
task_events_header_filtered = ['timestamp','event_type',
                      'priority','cpu_req','ram_req','disk_space_req','different_machine','task_key', 'time_taken']

In [8]:
count = 0
final_df = pd.DataFrame()

for name in tqdm(glob.glob(data_dir+"task_events/*.csv.gz")):
#     print(name)
    df = pd.read_csv(name, header=None)
    df.columns = task_events_header
    df['task_key'] = df['job_id'].astype(str) +'-'+ df['task_index'].astype(str) 
    df = df[pd.isnull(df['missing_info']) == True]
    df = df.loc[(df['event_type']==0) | (df['event_type']==1)]
    df.drop(
        [
            "user_name", "machine_id", "job_id", "task_index",
            "sched_class", "missing_info"
        ],
        axis=1,
        inplace=True)
    
    df.to_csv(name.replace(".csv", ".filtered.csv"), index=False, compression='gzip')
        
    count += 1
    if count >= 200:
        break
    
print (count, "files processed")


 40%|███████████████████████████████▊                                                | 199/500 [22:48<32:35,  6.50s/it]

200 files processed


In [12]:
count = 0
final_df = pd.DataFrame()

## better idea then iteration, groupby on the [task_key, event_type] and take max time_stamp
## then separate the event==0 and event==4 
## then join and calculate time taken

for name in tqdm(glob.glob(data_dir+"task_events/*.filtered.csv.gz")):
    print(name)
    df = pd.read_csv(name)
        
#     df = df.loc[df['event_type']==0]

    final_df = final_df.append(df)
    count += 1
    if count == 200:
        break
    
print (count, "files appended")

storename = data_dir+"task_events/"+"appended.h5"
print ("Writing the data to ", storename)

store = pd.HDFStore(storename)
store['df'] = final_df
store.close()

print ("Task Completed")


  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

./data/clusterdata-2011-2/task_events\part-00000-of-00500.filtered.csv.gz



  0%|▍                                                                                 | 1/200 [00:00<02:34,  1.29it/s]

./data/clusterdata-2011-2/task_events\part-00001-of-00500.filtered.csv.gz



  1%|▊                                                                                 | 2/200 [00:00<01:57,  1.69it/s]

...

./data/clusterdata-2011-2/task_events\part-00197-of-00500.filtered.csv.gz


 99%|███████████████████████████████████████████████████████████████████████████████▏| 198/200 [11:44<00:13,  6.53s/it]

./data/clusterdata-2011-2/task_events\part-00198-of-00500.filtered.csv.gz


100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [11:51<00:06,  6.54s/it]

./data/clusterdata-2011-2/task_events\part-00199-of-00500.filtered.csv.gz
200 files appended
Writing the data to  ./data/clusterdata-2011-2/task_events/appended.h5
Task Completed


In [17]:
final_df[final_df["task_key"] == '1005190908-4']

,timestamp,event_type,priority,cpu_req,ram_req,space_req,diff_machine,task_key
1165,0,0,10,0.04376,0.003975,0.000019,1.0,1005190908-4
145297,0,1,10,0.04376,0.003975,0.000019,1.0,1005190908-4
68874,354150400311,0,10,0.04376,0.003975,0.000019,1.0,1005190908-4
68938,354152925043,1,10,0.04376,0.003975,0.000019,1.0,1005190908-4
49026,693364066812,0,10,0.04376,0.003975,0.000019,1.0,1005190908-4
49034,693365736659,1,10,0.04376,0.003975,0.000019,1.0,1005190908-4
49343,703111693103,0,10,0.04376,0.003975,0.000019,1.0,1005190908-4
49355,703113545279,1,10,0.04376,0.003975,0.000019,1.0,1005190908-4
93397,824243905383,0,10,0.04376,0.003975,0.000019,1.0,1005190908-4
93822,824274506192,1,10,0.04376,0.003975,0.000019,1.0,1005190908-4


In [ ]:
final_df[['']]